## **Inference Model Hybrid Terbaik**

Uji coba model hybrid terbaik untuk ulasan pengguna aplikasi SIGNAL yang belum terlihat sama sekali oleh model hybrid.

In [17]:
!pip install transformers torch

## **Class Indonesian Roberta Feature Extractor (Arsitektur Model Hybrid)**

Indonesian Roberta di sini digunakan sebagai model yang bertugas untuk mengekstraksi fitur dari bentuk huruf ke dalam bentuk numerik atau representasi vektor. Repositori model nya dapat diakses di sini:

https://huggingface.co/w11wo/indonesian-roberta-base-sentiment-classifier

In [18]:
# Import library pemodelan
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

In [19]:
# Konfigurasi variabel global
# Runtime CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"

In [20]:
# Class untuk arsitektur model hybrid
class HybridRoberta(nn.Module):
    def __init__(self, model_name, n_classes, head_type, hidden_dim):
        super(HybridRoberta, self).__init__()

        self.head_type = head_type.lower()
        print(f"Menginisialisasi Model Hybrid: RoBERTa + {self.head_type.upper()}...")

        # Base Model (Feature Extractor)
        self.roberta = AutoModel.from_pretrained(model_name)

        # Freeze sebagian layer RoBERTa agar training lebih cepat & hemat memori (Opsional dan Kondisional)
        # Hapus komentarnya jika GPU Out of Memory
        # for param in self.roberta.parameters():
        #     param.requires_grad = False

        # Ukuran output RoBERTa base = 768
        self.roberta_dim = 768

        # Define Head (LSTM / GRU / CNN)
        if self.head_type == 'lstm':
            self.head = nn.LSTM(
                input_size=self.roberta_dim,
                hidden_size=hidden_dim,
                num_layers=1,
                batch_first=True,
                bidirectional=True
            )
            self.classifier = nn.Linear(hidden_dim * 2, n_classes) # *2 karena Bidirectional

        elif self.head_type == 'gru':
            self.head = nn.GRU(
                input_size=self.roberta_dim,
                hidden_size=hidden_dim,
                num_layers=1,
                batch_first=True,
                bidirectional=True
            )
            self.classifier = nn.Linear(hidden_dim * 2, n_classes)

        elif self.head_type == 'cnn':
            # CNN 1D untuk teks
            self.head = nn.Conv1d(
                in_channels=self.roberta_dim,
                out_channels=hidden_dim,
                kernel_size=3, # Filter size (melihat 3 kata sekaligus)
                padding=1
            )
            self.classifier = nn.Linear(hidden_dim, n_classes)

        else:
            raise ValueError("Tipe head harus 'lstm', 'gru', atau 'cnn'")

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, input_ids, attention_mask):
        # Feature Extraction dengan RoBERTa
        # output: (batch_size, seq_len, 768)
        roberta_out = self.roberta(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

        # Masuk ke Head (Hybrid)
        if self.head_type in ['lstm', 'gru']:
            # LSTM/GRU Output: (batch, seq_len, hidden*2)
            output, _ = self.head(roberta_out)
            # Mean Pooling: Ambil rata-rata semua token output
            output = torch.mean(output, dim=1)

        elif self.head_type == 'cnn':
            # CNN butuh input (batch, channels, seq_len), lakukan permute
            # roberta_out: (batch, 768, seq_len)
            roberta_out = roberta_out.permute(0, 2, 1)

            output = self.head(roberta_out)
            # output: (batch, hidden_dim, seq_len)

            # Global Max Pooling (Ambil fitur paling menonjol)
            output = F.max_pool1d(output, kernel_size=output.shape[2])
            # output: (batch, hidden_dim, 1) -> Squeeze jadi (batch, hidden_dim)
            output = output.squeeze(2)

        # Klasifikasi
        output = self.dropout(output)
        return self.classifier(output)

In [21]:
# Fungsi untuk load model hybrid terbaik untuk inference
def load_model_inference(model_path, head_type):
    print(f"Loading model: {head_type.upper()}...")
    MODEL_NAME = "w11wo/indonesian-roberta-base-sentiment-classifier"

    # Arsitektur Hybrid Model
    model = HybridRoberta(
        model_name=MODEL_NAME,
        n_classes=3,
        head_type=head_type,
        hidden_dim=256
    )

    # Isi dengan Bobot (.bin)
    # map_location=device (agar jalan di CPU)
    model.load_state_dict(torch.load(model_path, map_location=device))

    model = model.to(device)
    model.eval() # Set mode evaluasi

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    return model, tokenizer

## **Uji Coba Model Hybrid Terbaik dengan Data Baru**

In [22]:
# Fungsi klasifikasi sentimen ulasan aplikasi SIGNAL
def predict_sentiment(text, model, tokenizer):
    encoded_review = tokenizer(
        text,
        max_length=128,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    with torch.no_grad(): # Hemat memori
        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)

    class_names = ['Negatif', 'Netral', 'Positif']
    return class_names[prediction]

In [23]:
# Load Model
# Upload file .bin model hybrid terbaik (Roberta+GRU)
model_gru, tokenizer = load_model_inference('best_model_skema3_cnn.bin', head_type='cnn')

Loading model: CNN...
Menginisialisasi Model Hybrid: RoBERTa + CNN...


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: w11wo/indonesian-roberta-base-sentiment-classifier
Key                             | Status     | 
--------------------------------+------------+-
classifier.out_proj.weight      | UNEXPECTED | 
classifier.dense.bias           | UNEXPECTED | 
classifier.out_proj.bias        | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
classifier.dense.weight         | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [24]:
# Uji model hybrid terbaik dengan ulasan bebas
# Tes Prediksi untuk ulasan positif
review_pos = input("Masukkan ulasan aplikasi SIGNAL (Positif): ")
hasil_pos = predict_sentiment(review_pos, model_gru, tokenizer)

print(f"Review: {review_pos}")
print(f"Sentimen: {hasil_pos}")

Masukkan ulasan aplikasi SIGNAL (Positif): aplikasi nya ngebantu banget jadi saya ga perlu datang ke kantor, simpel tinggal lewat HP
Review: aplikasi nya ngebantu banget jadi saya ga perlu datang ke kantor, simpel tinggal lewat HP
Sentimen: Positif


In [25]:
# Tes Prediksi untuk ulasan negatif
review_neg = input("Masukkan ulasan aplikasi SIGNAL (Negatif): ")
hasil_neg = predict_sentiment(review_neg, model_gru, tokenizer)

print(f"Review: {review_neg}")
print(f"Sentimen: {hasil_neg}")

Masukkan ulasan aplikasi SIGNAL (Negatif): aplikasi lambat buang-buang waktu aja  Review: aplikasi lambat buang-buang waktu aja 
Review: aplikasi lambat buang-buang waktu aja  Review: aplikasi lambat buang-buang waktu aja 
Sentimen: Negatif


In [26]:
# Tes Prediksi untuk ulasan netral
review_net = input("Masukkan ulasan aplikasi SIGNAL (Netral): ")
hasil_net = predict_sentiment(review_net, model_gru, tokenizer)

print(f"Review: {review_net}")
print(f"Sentimen: {hasil_net}")

Masukkan ulasan aplikasi SIGNAL (Netral): bayar pajak online bisa lewat aplikasi SIGNAL Review: bayar pajak online bisa lewat aplikasi SIGNAL
Review: bayar pajak online bisa lewat aplikasi SIGNAL Review: bayar pajak online bisa lewat aplikasi SIGNAL
Sentimen: Netral
